In [27]:
import pandas as pd

In [28]:
yrbss = pd.read_csv('../data/yrbss_2017.csv')
us = yrbss[yrbss['LocationAbbr'] == 'XX']

In [29]:
def question_loop(topic, subtopic, risk_question, question_df):
    Gender = ['Male', 'Female']
    Race = ['Asian', 'Black or African American', 'Hispanic or Latino', 'White', 
        'Multiple Race', 'American Indian or Alaska Native', 'Native Hawaiian or Other Pacific Islander']
    Sexuality = ['Gay or lesbian', 'Bisexual', 'Not sure', 'Heterosexual (straight)']
    
    group = []
    data_val = []
    low_val = []
    high_val = []
    significance = []

    # The overall results for this question, all significance tests will go against these values
    q_total = question_df[(question_df['Grade'] == 'Total') & (question_df['Sex'] == 'Total') &
        (question_df['Race'] == 'Total') & (question_df['SexualIdentity'] == 'Total') & 
        (question_df['SexOfSexualContacts'] == 'Total')]
    q_val = q_total['Greater_Risk_Data_Value'].iloc[0]
    q_low = q_total['Greater_Risk_Low_Confidence_Limit'].iloc[0]
    q_high = q_total['Greater_Risk_High_Confidence_Limit'].iloc[0]
    
    for gender in Gender:
        web_total = question_df[(question_df['Grade'] == 'Total') & (question_df['Sex'] == gender) &
            (question_df['Race'] == 'Total') & (question_df['SexualIdentity'] == 'Total') & 
            (question_df['SexOfSexualContacts'] == 'Total')]
        g_val = web_total['Greater_Risk_Data_Value'].iloc[0]
        g_low = web_total['Greater_Risk_Low_Confidence_Limit'].iloc[0]
        g_high = web_total['Greater_Risk_High_Confidence_Limit'].iloc[0]
        if g_val > q_high and q_val < g_low:
            group.append(gender)
            data_val.append(g_val)
            low_val.append(g_low)
            high_val.append(g_high)
            significance.append('more')
    for race in Race:
        web_total = question_df[(question_df['Grade'] == 'Total') & (question_df['Sex'] == 'Total') &
            (question_df['Race'] == race) & (question_df['SexualIdentity'] == 'Total') & 
            (question_df['SexOfSexualContacts'] == 'Total')]
        g_val = web_total['Greater_Risk_Data_Value'].iloc[0]
        g_low = web_total['Greater_Risk_Low_Confidence_Limit'].iloc[0]
        g_high = web_total['Greater_Risk_High_Confidence_Limit'].iloc[0]
        if g_val > q_high and q_val < g_low:
            group.append(race)
            data_val.append(g_val)
            low_val.append(g_low)
            high_val.append(g_high)
            significance.append('more')
    for sexuality in Sexuality:
        web_total = question_df[(question_df['Grade'] == 'Total') & (question_df['Sex'] == 'Total') &
            (question_df['Race'] == 'Total') & (question_df['SexualIdentity'] == sexuality) & 
            (question_df['SexOfSexualContacts'] == 'Total')]
        g_val = web_total['Greater_Risk_Data_Value'].iloc[0]
        g_low = web_total['Greater_Risk_Low_Confidence_Limit'].iloc[0]
        g_high = web_total['Greater_Risk_High_Confidence_Limit'].iloc[0] 
        if g_val > q_high and q_val < g_low:
            group.append(sexuality)
            data_val.append(g_val)
            low_val.append(g_low)
            high_val.append(g_high)
            significance.append('more')
            
    question_results_df = pd.DataFrame.from_dict({'topic': topic, 'subtopic': subtopic, 
            'question': risk_question, 'group': group, 'data_val': data_val, 
            'low_val': low_val, 'high_val': high_val, 'significance': significance})
    return question_results_df

In [30]:
question_results_df = pd.DataFrame()
for topic in us['Topic'].value_counts().items():
    #print('topic', topic)
    topic_df = us[us['Topic'] == topic[0]]
    for subtopic in topic_df['Subtopic'].value_counts().items():
        #print('subtopic', subtopic)
        subtopic_df = topic_df[topic_df['Subtopic'] == subtopic[0]]
        for risk_question in subtopic_df['Greater_Risk_Question'].value_counts().items():
            #print('risk_question', risk_question)
            risk_question_df = subtopic_df[subtopic_df['Greater_Risk_Question'] == risk_question[0]]
            results_df = question_loop(topic[0], subtopic[0], risk_question[0], risk_question_df)
            question_results_df = question_results_df.append(results_df)
question_results_df.shape

(341, 8)

In [31]:
# Switch group list to create the dataframe for each of the group types
Group = ['Gay or lesbian', 'Bisexual', 'Not sure', 'Heterosexual (straight)']

# ['Asian', 'Black or African American', 'Hispanic or Latino', 'White', 
#         'Multiple Race', 'American Indian or Alaska Native', 
#          'Native Hawaiian or Other Pacific Islander']

# ['Male', 'Female']

topics = list(question_results_df['topic'].unique())
group_count_df = pd.DataFrame(0, index=topics, columns=Group)
for topic in question_results_df['topic'].unique():
    topic_df = question_results_df[(question_results_df['topic'] == topic)]
    for group in Group:
        group_count_df.at[topic,group] = len(topic_df[(topic_df['group'] == group)])
group_count_df.to_csv('../data/data_df.csv')